In [12]:
from keras import datasets, models, layers, utils, Sequential
import numpy as np
from keras.src.layers import Dense
from keras_tuner.tuners import RandomSearch,Hyperband,BayesianOptimization

In [13]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [14]:
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000, 784)
x_train=x_train/255
x_test=x_test/255
y_train=utils.to_categorical(y_train,10)
y_test=utils.to_categorical(y_test,10)

In [15]:
def build_model(hp):
    model = Sequential()
    activation_choice=hp.Choice('activation_choice',['relu','sigmoid','tanh','elu','selu'])
    model.add(Dense(units=hp.Int('units_input', min_value=512, max_value=1024, step=32), activation=activation_choice,input_dim=784))
    model.add(Dense(units=hp.Int('units_hidden',min_value=128, max_value=600, step=32), activation=activation_choice))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer=hp.Choice('optimizer',values=['adam','rmsprop','SGD']), metrics=['accuracy'])
    return model

In [16]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=80,directory='test_directory')
tuner.search_space_summary()

Search space summary
Default search space size: 4
activation_choice (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


C:\Users\administrator\tensorflow\pythonProject1\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(x_train,y_train,batch_size=256,epochs=20,validation_split=0.2)

Trial 65 Complete [00h 00m 45s]
val_accuracy: 0.8565833568572998

Best val_accuracy So Far: 0.8989999890327454
Total elapsed time: 01h 06m 08s

Search: Running Trial #66

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation_choice
960               |768               |units_input
512               |192               |units_hidden
rmsprop           |adam              |optimizer

Epoch 1/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.6704 - loss: 0.9541 - val_accuracy: 0.8113 - val_loss: 0.5293
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.8348 - loss: 0.4512 - val_accuracy: 0.8561 - val_loss: 0.3898
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.8562 - loss: 0.3819 - val_accuracy: 0.8662 - val_loss: 0.3659
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.8723 - loss: 0.3341 - val_accuracy: 0.8695 - val_loss: 0.3583
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step -

In [ ]:
tuner.results_summary()

In [ ]:
models=tuner.get_best_models(num_models=3)
for model in models:
    model.summary()
    model.evaluate(x_test,y_test)
    print()